# 🦊 贝叶斯优化

相较于随机网格搜索、对半网格搜索，贝叶斯过程带有先验过程。

### 1. 贝叶斯优化基本流程

1. 定义需要估计的f(x)及x的定义域
2. 取出有限的n个x，求解对应f(x)观测值
3. 根据有限的观测值，对函数估计（先验知识），得出该估计f上的目标值
4. 定义某种规则，以确定下一个需要计算的观测点

持续在2-4步骤中循环，直至假设分布上的目标值达到标准，或者所有计算资源被用完为止

>1. 当贝叶斯优化不用于HPO时，一般f(x)是完全的黑盒函数
>2. 在HPO中，自变量x就是超参数空间
>3. 根据有限观测值、对函数分布进行估计的工具被称为概率代理模型。自带某些假设，可以根据有限个点估计出函数分布。默认使用基于搞死混合模型的TPE过程

### 2. 贝叶斯优化的实现

可以实现的库很多，如：https://www.automl.org/hpo-overview/hpo-tools/hpo-packages/

| HPO库     | 优劣                                                         | 推荐指数 |
| --------- | ------------------------------------------------------------ | -------- |
| bayes_opt | `pip install bayesian-optimization`<br />实现基于高斯过程的贝叶斯优化<br />当参数空间由大量连续性参数构成时推荐<br /><br />包含大量离散型参数时避免使用<br />算力/时间稀缺时避免使用 | 2        |
| hyperopt  | `pip install hyperopt`<br />实现基于TPE的贝叶斯优化<br />支持各类提效工具<br />进度条清晰，展示美观，较少怪异警告或报错<br />可推广至深度学习领域<br /><br />不支持基于高斯过程的贝叶斯优化<br />代码限制多、较为复杂、灵活性差 | 4        |
| optuna    | `pip install optuna`<br />可实现基于各类算法的贝叶斯优化<br />代码最简洁，具有一定的灵活性<br />可推广至深度学习领域<br /><br />非关键功能维护不佳，有怪异警告与报错 | 4        |
| Skopt     | `pip install scikit-optimize`<br />作为Optuna辅助包          |          |

## 贰丨Bayes_Opt实现



主要流程：

第一步，定义目标函数的模板

```python
def objective():
    # 评估器
    reg = RandomForestRegressor()
    
    # 交叉验证
    cv = KFold()
    result = cross_validate(reg,cv)
    
    # 交叉验证结果
    loss = result['test_rmse']    
    
    return loss
```




>1. 目标函数的输入必须是具体的超参数，而不是整个超参数空间，更不能是数据、算法等超参数外的元素。在定义目标函数时，需要让超参数作为目标函数输入
>
>2. 超参数输入只能是浮点数，不支持整数与字符串。
>
>3. Bayes_Opt只支持寻找f(x)最大值，不支持寻找最小值

In [9]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_validate,KFold
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import time

In [2]:
# 导入加利福尼亚房价数据
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

data = fetch_california_housing()
print(data.keys())

# 划分数据集
x = data['data']
y = data['target']
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=22)

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])


In [5]:
# 第一步，定义目标函数
def bayesopt_objective(n_estimators, max_depth, max_features, min_impurity_decrease):
    # 定义评估器
    # 需要调整的超参数等于目标函数的输入，不需要调整的超参数直接等于固定值
    # 默认参数输入一定是浮点数，因此需要套上int函数处理成整数
    reg = RandomForestRegressor(n_estimators=int(n_estimators),
                                max_depth=int(max_depth),
                                max_features=int(max_features),
                                min_impurity_decrease=min_impurity_decrease,
                                random_state=22,
                                verbose=False,
                                n_jobs=-1)

    # 定义损失的输出，5折交叉验证下的结果，输出负根均方误差（-RMSE）
    # 交叉验证需要使用数据，但不能让数据xy成为目标函数的输入
    cv = KFold(n_splits=5, shuffle=True, random_state=22)
    validation_loss = cross_validate(reg, train_x, train_y, scoring='neg_root_mean_squared_error', cv=cv,
                                     error_score='raise')

    # 交叉验证输出的评估指标是负根均方误差，因此本来就是负值
    # 目标函数可以直接输出改损失的均值
    return np.mean(validation_loss['test_score'])

第二步，定义参数空间

使用字典方式定义参数空间，取值范围为双向闭区间

Bayes_Opt只支持填写参数的上下界，不支持填写步长等参数，且会讲所有参数都当做连续性超参数处理，因此Bayes_Opt会直接取出闭区间中任意浮点数作为备选参数（因此比其他贝叶斯优化库更大、更密，需要的迭代次数更多）

In [6]:
param_grid_simple = {'n_estimators': (80, 100),
                     'max_depth': (10, 25),
                     'max_features': (10, 20),
                     'min_impurity_decrease': (0, 1)}

第三步，定义优化目标函数的具体流程

在贝叶斯优化的实践中，都有涉及随机性的过程。在大部分优化库中，随机性无法控制。（即便是控制随机数种子，也无法固定优化过程，但选择出的超参数是可以复现的）

In [7]:
def param_bayes_opt(init_points, n_iter):
    # 定义优化器，先实例化优化器
    opt = BayesianOptimization(bayesopt_objective,  # 目标函数
                               param_grid_simple,  # 备选参数空间
                               random_state=22  # 随机数种子(虽然无法控制过程)
                               )
    
    # 使用优化器，Bayes_Opt只支持最大值
    opt.maximize(init_points=init_points, # 抽取多少个初始观测值
                 n_iter=n_iter # 一共观测/迭代多少次
                 )
    
    # 优化完成，取出最佳参数和最佳分数
    param_best = opt.max['params']
    score_best = opt.max['target']
    
    # 打印最佳参数与最佳分数
    print(f'Best Params: {param_best}\n'
          f'Best CVScore: {score_best}')
    
    return param_best,score_best

第四步，定义验证函数（非必须）

在贝叶斯优化中，目标函数中交叉验证即数据分割都是可以规定好的，因此目标函数中设置了随机数种子，贝叶斯优化给出的最佳分数一定是与验证后分数相同

In [8]:
def bayes_opt_validation(params_best):
    reg = RandomForestRegressor(n_estimators=int(params_best['n_estimators']))
    
    cv = KFold(n_splits=5,shuffle=True,random_state=22)
    validation_loss = cross_validate(reg, train_x,train_y,scoring='neg_root_mean_squared_error', cv=cv,verbose=False,njobs=-1)
    return np.mean(validation_loss['test_score'])

第五步，执行优化流程

In [10]:
start = time.time()
params_best,score_best = param_bayes_opt(20,280)
print(f'耗时: {(time.time()-start)/60} min')
# validation_score = bayes_opt_validation(params_best)
# print(f'validation_score: {validation_score}')

|   iter    |  target   | max_depth | max_fe... | min_im... | n_esti... |
-------------------------------------------------------------------------
| 1         | -1.035    | 13.13     | 14.82     | 0.4205    | 97.18     |
| 2         | -0.9542   | 12.57     | 13.39     | 0.2705    | 93.82     |
| 3         | -0.7283   | 13.31     | 18.12     | 0.01053   | 91.22     |
| 4         | -0.9544   | 22.21     | 17.45     | 0.1891    | 80.12     |
| 5         | -1.157    | 21.58     | 19.58     | 0.7019    | 85.95     |
| 6         | -0.9588   | 21.52     | 16.88     | 0.3872    | 92.3      |
| 7         | -1.157    | 16.41     | 15.84     | 0.7026    | 82.24     |
| 8         | -1.157    | 23.85     | 19.89     | 0.6774    | 95.9      |
| 9         | -1.157    | 10.44     | 11.78     | 0.8749    | 94.9      |
| 10        | -1.157    | 17.62     | 11.28     | 0.4843    | 90.32     |
| 11        | -1.116    | 14.67     | 10.46     | 0.4377    | 81.92     |
| 12        | -1.157    | 10.68     | 